# Loading Dataset

In [334]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [335]:
df =  pd.read_csv('datasets/nieruchomosci-online_dataset_raw.csv')

In [336]:
df.head(10)

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899 000 zł,"51,70 m²","17 388,78 zł/m²",1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1 575 000 zł,125 m²,12 600 zł/m²,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1 250 000 zł,"65,23 m²","19 162,96 zł/m²",2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740 250 zł,"49,35 m²",15 000 zł/m²,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1 200 000 zł,"71,06 m²","16 887,14 zł/m²",parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1 198 680 zł,"71,35 m²",16 800 zł/m²,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",625 000 zł,40 m²,15 625 zł/m²,2,3.0,2,2000,garaż,wtórny,własność
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie",899 000 zł,"74,17 m²","12 120,80 zł/m²",2,5.0,3,2003,parking publiczny / na ulicy,wtórny,NaN
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",1 050 000 zł,60 m²,17 500 zł/m²,3,4.0,3,2023,w garażu podziemnym,wtórny,"własność, księga wieczysta"
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599 000 zł,40 m²,14 975 zł/m²,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"


In [337]:
df.shape

(3949, 13)

# Data Cleaning

Steps:
* feature name change
* area and price from string to float + metrics
* missing values
* feature type change

# Feature name change

In [338]:
df.rename(columns={'area': 'area (m^2)'}, inplace=True)
df.rename(columns={'price': 'price (zł)'}, inplace=True)
df.rename(columns={'no of floors/stores in the building': 'number of floors'}, inplace=True)
df.rename(columns={'floor/store': 'floor number'}, inplace=True)

In [339]:
df['price (zł)'] = df['price (zł)'].str.replace(',', '')

# Rozdzielenie wartości w kolumnie "price (zł)" na dwie nowe kolumny
price_split = df['price (zł)'].str.extract(r'([\d\s]+)\s+(.*)')
df['price_amount'] = price_split[0].str.replace(' ', '').astype(float)
df['currency'] = price_split[1]

# Zamiana przecinka na kropkę i konwersja na float w kolumnie "area (m^2)"
df['area_amount'] = df['area (m^2)'].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)').astype(float)

# Wyciągnięcie jednostki metryki z kolumny "area (m^2)"
df['area_unit'] = df['area (m^2)'].str.extract(r'([a-zA-Z²]+)')

# Usunięcie wcześniejszych kolumn
df.drop(columns=['price (zł)', 'area (m^2)'], inplace=True)

In [340]:
unique_area_units = df['area_unit'].unique()
print("Unikalne jednostki metryczne:", unique_area_units)

unique_currencies = df['currency'].unique()
print("Unikalne waluty:", unique_currencies)

Unikalne jednostki metryczne: ['m²']
Unikalne waluty: ['zł' nan '€']


In [341]:
EURO_RATE = 4.3

# Warunkowe mnożenie kwoty przez stałą, jeśli waluta to euro (nie optymalne, ale za głupi na to jestem)
df['price_amount'] = df.apply(lambda row: row['price_amount'] * EURO_RATE if row['currency'] == 'euro' else row['price_amount'], axis=1)

# Zmiana waluty na "zł"
df.loc[df['currency'] == '€', 'currency'] = 'zł'

In [342]:
unique_area_units = df['area_unit'].unique()
print("Unikalne jednostki metryczne:", unique_area_units)

unique_currencies = df['currency'].unique()
print("Unikalne waluty:", unique_currencies)

Unikalne jednostki metryczne: ['m²']
Unikalne waluty: ['zł' nan]


In [343]:
df.head(20)

,url,name/title,address,price-per-area,floor number,number of floors,no of rooms,year of construction,parking space,market,form of ownership,price_amount,currency,area_amount,area_unit
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie","17 388,78 zł/m²",1,3.0,2,1980,tak,wtórny,własność,899000.0,zł,51.70,m²
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",12 600 zł/m²,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta",1575000.0,zł,125.00,m²
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie","19 162,96 zł/m²",2,5.0,3,2023,tak,wtórny,NaN,1250000.0,zł,65.23,m²
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",15 000 zł/m²,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN,740250.0,zł,49.35,m²
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie","16 887,14 zł/m²",parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta",1200000.0,zł,71.06,m²
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",16 800 zł/m²,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność,1198680.0,zł,71.35,m²
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",15 625 zł/m²,2,3.0,2,2000,garaż,wtórny,własność,625000.0,zł,40.00,m²
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie","12 120,80 zł/m²",2,5.0,3,2003,parking publiczny / na ulicy,wtórny,NaN,899000.0,zł,74.17,m²
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",17 500 zł/m²,3,4.0,3,2023,w garażu podziemnym,wtórny,"własność, księga wieczysta",1050000.0,zł,60.00,m²
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",14 975 zł/m²,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta",599000.0,zł,40.00,m²


In [344]:
unique_area_units = df['form of ownership'].unique()
print("Unikalne jednostki metryczne:", unique_area_units)

unique_currencies = df['market'].unique()
print("Unikalne waluty:", unique_currencies)

Unikalne jednostki metryczne: ['własność' 'własność, księga wieczysta' nan
 'spółdzielcze własnościowe, księga wieczysta'
 'pierwotny (oferta dewelopera)' 'udział w KW' 'spółdzielcze własnościowe'
 'udział ze wskazaniem, KW na budynku' 'księga wieczysta' 'inna' 'udział'
 'inna, księga wieczysta' 'Udział ze wskazaniem' 'Udział'
 'udział, księga wieczysta' 'udział ze wskazaniem' 'Chce dopłacić' 'tak']
Unikalne waluty: ['wtórny' 'pierwotny (zobacz inne nowe mieszkania w Krakowie)' 'dostępne'
 'II kwartał 2024' 'III kwartał 2024' 'IV kwartał 2022, oddana do użytku'
 'II kwartał 2025' 'III kwartał 2025' 'październik 2023, zrealizowana'
 'czerwiec 2023, zrealizowana' 'II kwartał 2023, oddana do użytku'
 'sierpień 2024' 'październik 2025' 'IV kwartał 2024' nan 'zarezerwowane'
 'sprzedane' 'I kwartał 2024' 'I kwartał 2025' 'IV kwartał 2025'
 'lipiec 2025' '2025' 'III kwartał 2023, oddana do użytku'
 'kwiecień 2023, oddana do użytku' 'marzec 2024' '1 miesiąc' 'natychmiast']


## Missing Values

| Attribute               | Value |
|-------------------------|-------|
| url                     | 0     |
| name/title              | 0     |
| address                 | 101   |
| price-per-area          | 29    |
| floor number            | 0     |
| number of floors        | 239   |
| no of rooms             | 0     |
| year of construction    | 0     |
| parking space           | 0     |
| market                  | 31    |
| form of ownership       | 2461  |
| price_amount            | 29    |
| currency                | 29    |
| area_amount             | 0     |
| area_unit               | 0     |

In [345]:
# Nowe obliczenie price-per-area bo tamtemu nie ufam
df['price_per_area'] = df['price_amount'] / df['area_amount']
df.drop(columns=['price-per-area'], inplace=True)

In [346]:
# Wywalam wszystkie kolumny, w których nie mam danych odnośnie ceny
missing_currency = df[df['currency'].isnull()]
missing_currency

,url,name/title,address,floor number,number of floors,no of rooms,year of construction,parking space,market,form of ownership,price_amount,currency,area_amount,area_unit,price_per_area
26,https://wieliczka.nieruchomosci-online.pl/nowe...,Nowe mieszkanie ul. Magnoliowa,NaN,wrzesień 2024,NaN,1,3,-,dostępne,NaN,NaN,NaN,57.83,m²,NaN
27,https://wieliczka.nieruchomosci-online.pl/nowe...,Nowe mieszkanie ul. Magnoliowa,NaN,wrzesień 2024,NaN,1,3,naziemne,dostępne,NaN,NaN,NaN,57.83,m²,NaN
28,https://wieliczka.nieruchomosci-online.pl/nowe...,Nowe mieszkanie ul. Magnoliowa,NaN,wrzesień 2024,NaN,1,3,naziemne,dostępne,NaN,NaN,NaN,57.83,m²,NaN
45,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,III kwartał 2024,NaN,suterena,3,garaż,dostępne,NaN,NaN,NaN,58.36,m²,NaN
172,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Kleparz, ul. Długa 24",NaN,luty 2024,NaN,4,2,garaż,dostępne,NaN,NaN,NaN,39.84,m²,NaN
175,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Kleparz, ul. Długa 24",NaN,luty 2024,NaN,4,2,garaż,dostępne,NaN,NaN,NaN,57.45,m²,NaN
177,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Kleparz, ul. Długa 24",NaN,luty 2024,NaN,4,2,garaż,dostępne,NaN,NaN,NaN,38.93,m²,NaN
220,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,wrzesień 2024,NaN,1,4,naziemne,dostępne,NaN,NaN,NaN,93.00,m²,NaN
221,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,wrzesień 2024,NaN,parter,3,naziemne,zarezerwowane,NaN,NaN,NaN,55.00,m²,NaN
226,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Olszyny, ul. Petrażyckiego",NaN,wrzesień 2024,NaN,1,4,naziemne,dostępne,NaN,NaN,NaN,93.00,m²,NaN


In [347]:
# Brak przesunięcia w danych z missing price więc usuwam wszystkie, które nie posiadają wyceny
missing_currency['address']

26                                             NaN
27                                             NaN
28                                             NaN
45                                             NaN
172                                            NaN
175                                            NaN
177                                            NaN
220                                            NaN
221                                            NaN
226                                            NaN
257                                            NaN
273                                            NaN
274                                            NaN
276                                            NaN
304                                            NaN
305                                            NaN
309                                            NaN
318                                            NaN
321                                            NaN
323                            

In [348]:
df.dropna(subset=['price_amount'], inplace=True)

In [349]:
# Wywalam całą kolumnę form of ownership bo ma ponad 2500 pustych wierszy
df.drop(columns=['form of ownership'], inplace=True)

In [350]:
# Z racji tego, że brakuje tylko 29 marketów na prawie 4000 mieszkań to braki uzupełnię średnia.
# TODO, może się tak zdażyć, że floor number będzie większe niż number of floors, więc trzeba też napisać kod, który zmienia tą wartość jeśli floor number jest większe

number_of_floors_na = df['number of floors'].unique()
print("Unikalne wartości w number of floors:", number_of_floors_na)

Unikalne wartości w number of floors: [ 3.  4.  5. 10.  1.  2.  7.  6.  9. nan  8. 14. 11. 15. 16. 13. 12. 45.]


In [351]:
mean_number_of_floors = round(df['number of floors'].mean())
fill_value = {'number of floors': mean_number_of_floors}
df.fillna(value=fill_value, inplace=True)

print("Średnia wartość w mean number of floors:", mean_number_of_floors)

Średnia wartość w mean number of floors: 5


In [352]:
# TODO, TU JEST PROBLEM (którego rozwiązania teraz nie wymyśle WIĘC żeby ruszyć dalej pozbywam się tej kolumny. (prawdopodobnie się jej pozbędziemy)

market = df['market'].unique()
print("Unikalne jednostki metryczne:", market)

df.drop(columns=['market'], inplace=True)

Unikalne jednostki metryczne: ['wtórny' 'pierwotny (zobacz inne nowe mieszkania w Krakowie)'
 'II kwartał 2024' 'III kwartał 2024' 'IV kwartał 2022, oddana do użytku'
 'II kwartał 2025' 'III kwartał 2025' 'październik 2023, zrealizowana'
 'czerwiec 2023, zrealizowana' 'II kwartał 2023, oddana do użytku'
 'sierpień 2024' 'październik 2025' 'IV kwartał 2024' nan 'I kwartał 2024'
 'I kwartał 2025' 'IV kwartał 2025' 'lipiec 2025' '2025'
 'III kwartał 2023, oddana do użytku' 'kwiecień 2023, oddana do użytku'
 'marzec 2024' '1 miesiąc' 'natychmiast']


In [353]:
df.isnull().sum()

url                      0
name/title               0
address                 74
floor number             0
number of floors         0
no of rooms              0
year of construction     0
parking space            0
price_amount             0
currency                 0
area_amount              0
area_unit                0
price_per_area           0
dtype: int64

## Shifted data / String data

In [358]:
unique_parking = df['parking space'].unique()
print("Unikalne wartości 'parking':", unique_parking)

Unikalne waluty: ['tak' 'w garażu podziemnym' 'parking publiczny / na ulicy'
 'garaż w bryle budynku' 'garaż' '-' 'przynależne na ulicy'
 'przynależne na terenie ogrodzonym' 'możliwość wykupienia' 'naziemne'
 'garaż wolnostojący' 'parking strzeżony w pobliżu' 'wiata garażowa']


In [359]:
# TODO, dokonuję uproszczeń, nie wiem czy tak akceptujecie

parking_counts = df['parking space'].value_counts()
parking_counts

parking space
-                                    965
w garażu podziemnym                  590
tak                                  572
parking publiczny / na ulicy         564
garaż                                439
przynależne na ulicy                 264
garaż wolnostojący                   161
garaż w bryle budynku                116
przynależne na terenie ogrodzonym     97
parking strzeżony w pobliżu           84
możliwość wykupienia                  39
naziemne                              17
wiata garażowa                        12
Name: count, dtype: int64

## Address from Title

# Feature Engineering